# R&S Experiment Results Summary

This notebook reads the PCS results saved as CSVs and prints them together.

In [1]:
import os
from pathlib import Path
import pandas as pd
import numpy as np

# --------- Config ---------
# Set this to the folder that contains the 'data' directory.
# Example: BASE_DIR = Path(r"D:/QRGMM/Esophageal_Cancer_Simulator/Treatment_Effects")
BASE_DIR = Path('.')

RESULTS_DIR = BASE_DIR / 'data' / 'RS' / 'results'
RESULTS_DIR

WindowsPath('data/RS/results')

In [2]:
def read_scalar_csv(path: Path):
    """Read a 1x1 numeric value from a csv written by MATLAB csvwrite."""
    if not path.exists():
        return np.nan
    # MATLAB csvwrite produces plain numbers; pd.read_csv is robust
    try:
        v = pd.read_csv(path, header=None).values
        # flatten and take first numeric
        v = np.array(v, dtype=float).ravel()
        return float(v[0]) if v.size else np.nan
    except Exception as e:
        print(f"Failed to read {path}: {e}")
        return np.nan

result_files = {
    'TS Procedure': 'PCS_TS.csv',
    'KN + Simulator': 'PCS_KN_Sim.csv',
    'QRGMM': 'PCS_RS_QRGMM.csv',
    'CWGAN': 'PCS_RS_CWGAN.csv',
    'Diffusion': 'PCS_RS_Diffusion.csv',
    'RectFlow': 'PCS_RS_RectFlow.csv',
}

rows = []
for method, fname in result_files.items():
    path = RESULTS_DIR / fname
    rows.append({
        'Method': method,
        'File': str(path.as_posix()),
        'PCS': read_scalar_csv(path),
        'Exists': path.exists(),
    })

df = pd.DataFrame(rows)
df

,Method,File,PCS,Exists
0,TS Procedure,data/RS/results/PCS_TS.csv,0.9946,True
1,KN + Simulator,data/RS/results/PCS_KN_Sim.csv,0.9952,True
2,QRGMM,data/RS/results/PCS_RS_QRGMM.csv,0.9858,True
3,CWGAN,data/RS/results/PCS_RS_CWGAN.csv,0.7480,True
4,Diffusion,data/RS/results/PCS_RS_Diffusion.csv,0.9830,True
5,RectFlow,data/RS/results/PCS_RS_RectFlow.csv,0.9716,True


In [3]:
# Pretty print
pd.set_option('display.max_colwidth', 120)

missing = df[~df['Exists']]
if not missing.empty:
    print('⚠️ Missing files (check your BASE_DIR / folder names):')
    display(missing[['Method','File']])

print('✅ Results summary:')
display(df[['Method','PCS']].sort_values('Method'))

# Also print as plain text (copy-paste friendly)
print('\n--- Plain text ---')
for _, r in df.iterrows():
    v = r['PCS']
    v_str = 'NaN' if (v is None or (isinstance(v, float) and np.isnan(v))) else f"{v:.6g}"
    print(f"{r['Method']}: {v_str}")

✅ Results summary:


,Method,PCS
3,CWGAN,0.7480
4,Diffusion,0.9830
1,KN + Simulator,0.9952
2,QRGMM,0.9858
5,RectFlow,0.9716
0,TS Procedure,0.9946



--- Plain text ---
TS Procedure: 0.9946
KN + Simulator: 0.9952
QRGMM: 0.9858
CWGAN: 0.748
Diffusion: 0.983
RectFlow: 0.9716


In [4]:
# Optional: save a combined summary table
OUT_PATH = RESULTS_DIR / 'PCS_summary_all_methods.csv'
df[['Method','PCS']].to_csv(OUT_PATH, index=False)
print('Saved:', OUT_PATH.resolve())

Saved: E:\project\GMM\QRGMM_vs_CWGANGP_revise_v2\QRGMM\Esophageal_Cancer_Simulator\Probability_of_Correct_Selection\data\RS\results\PCS_summary_all_methods.csv
